# Train, promote and deploy Boston house prices prediction model

This notebook contains steps and code to demonstrate support of AI Lifecycle features in Cloud Pak for Data. 
It contains steps and code to work with [`cpdctl`](https://github.com/IBM/cpdctl) CLI tool available in IBM github repository. 
It also introduces commands for getting model and training data, persisting model, deploying model
and promoting it between deployment spaces.

Some familiarity with Python is helpful. This notebook uses Python 3.7.


In [1]:
import base64
import json
import os
import platform
import requests
import tarfile
import zipfile
from IPython.core.display import display, HTML

## CPD Credentials
**Note**: when running this notebook inside IBM Cloud Pak for Data (CP4D) cluster, cpdctl takes advantage of [zero-configuration mode](https://github.com/IBM/cpdctl#zero-configuration) which means it can connect to the CP4D without explicit configuration. In that case the cells below that set credential and URL variables as well as cells that run `cpdctl config ...` commands can be skipped.

In [2]:
CPD_USER_NAME = 'YOUR CPD USER NAME'
CPD_USER_PASSWORD = 'YOUR CPD USER PASSWORD'
CPD_URL = 'YOUR CPD CLUSTER URL'

### Install the latest version of `cpdctl`

In [3]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPD_CONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [4]:
%%capture

%env PATH={CWD}:{PATH}
%env CPD_CONFIG={CPD_CONFIG}

In [5]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPD_CONFIG and os.path.exists(CPD_CONFIG):
    os.remove(CPD_CONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.0.105


### Add CPD user and profile configuration

Add "cpd_user" user to the `cpdctl` configuration

In [6]:
! cpdctl config user set cpd_user --username {CPD_USER_NAME} --password {CPD_USER_PASSWORD}

Add "cpd" profile to the `cpdctl` configuration

In [7]:
! cpdctl config profile set cpd --url {CPD_URL} --user cpd_user

List available profiles

In [1]:
! cpdctl config profile list

Name   Type      User       URL                                              Current
cpd    private   cpd_user   https://cpd-zen.apps.wp463case.cp.fyre.ibm.com   *


In [2]:
! cpdctl config profile use cpd

Switched to profile "cpd".


List available projects in current profile

In [11]:
! cpdctl project list

...
ID                                     Name                               Created                    Description                                          Tags   
47c57769-9876-46bc-ad26-259c559cbc3f   Clustering Demo (git-integrated)   2021-06-21T10:59:35.265Z   Demo showing how to cluster data in CPD using cod…   []   
68d739dd-7c54-41ae-9ed3-f04db182e559   jm-proj                            2021-07-15T10:27:38.452Z                                                        []   
87d3b6e0-19e1-4e4e-8c4c-11f1a9df789c   fvt-project                        2021-06-28T12:46:42.246Z                                                        []   
a5d7960c-5194-4013-9a0c-4da2d3f6583e   test-mo                            2021-06-23T11:43:56.992Z                                                        []   
e655370d-4513-45f9-b4c6-c45692044773   cpdctl-demo                        2021-07-15T09:28:43.603Z                                                        []   
fc5888cd-89d8-411b-b24a-ea1b9c5fd8

### Access the selected project assets

Get cpdctl-demo project ID and show details

In [12]:
result = ! cpdctl project list --output json --raw-output --jmes-query 'resources[4].metadata.guid'
PROJECT_ID = result.s

In [13]:
! cpdctl project get --project-id {PROJECT_ID}

...
                  
ID:            e655370d-4513-45f9-b4c6-c45692044773   
Name:          cpdctl-demo   
Created:       2021-07-15T09:28:43.603Z   
Description:      
Tags:          []   


Get project details in JSON format and extract it's name

In [14]:
! cpdctl project get --project-id {PROJECT_ID} --output json

{
  "entity": {
    "creator": "admin",
    "creator_iam_id": "1000330999",
    "description": "",
    "name": "cpdctl-demo",
    "public": false,
    "scope": {
      "bss_account_id": "999",
      "enforce_members": true
    },
    "storage": {
      "guid": "451b5035-8dde-4684-96fc-a4c88aa77711",
      "type": "assetfiles"
    }
  },
  "metadata": {
    "created_at": "2021-07-15T09:28:43.603Z",
    "guid": "e655370d-4513-45f9-b4c6-c45692044773",
    "updated_at": "2021-07-15T09:28:44.737Z",
    "url": "/v2/projects/e655370d-4513-45f9-b4c6-c45692044773"
  }
}


In [15]:
result = ! cpdctl project get --project-id {PROJECT_ID} --output json --jmes-query "entity.name" --raw-output
PROJECT_NAME = result.s
print("'{}' project ID is: {}".format(PROJECT_NAME, PROJECT_ID))

'cpdctl-demo' project ID is: e655370d-4513-45f9-b4c6-c45692044773


List assets in the project

In [16]:
! cpdctl asset search --project-id {PROJECT_ID} --type-name asset --query "*:*"

...
ID                                     Name                       Created                    Description   Type         State       Tags            Size   
91248743-3524-4b0f-878d-3aeef88c5fa5   Sample notebook            2021-07-15T09:29:11.000Z                 notebook     available   [notebook]      207   
bbeea775-b464-4abf-b2a2-17c31970cc47   credit_risk_training.csv   2021-07-15T10:19:26.000Z                 data_asset   available   [cpdctl-demo]   689222   


### Download data asset

Get "credit_risk_training.csv" data asset ID

In [17]:
result = ! cpdctl asset search --project-id {PROJECT_ID} --type-name data_asset --query "asset.name:credit_risk_training.csv" --output json --jmes-query "results[0].metadata.asset_id" --raw-output
DATA_ASSET_ID = result.s
print("'credit_risk_training.csv' data asset ID is: {}".format(DATA_ASSET_ID))

'credit_risk_training.csv' data asset ID is: bbeea775-b464-4abf-b2a2-17c31970cc47


Download data asset

In [18]:
! cpdctl asset get --project-id {PROJECT_ID} --asset-id {DATA_ASSET_ID}

...
                  
ID:            bbeea775-b464-4abf-b2a2-17c31970cc47   
Name:          credit_risk_training.csv   
Created:       2021-07-15T10:19:26.000Z   
Description:      
Type:          data_asset   
State:         available   
Tags:          [cpdctl-demo]   
Size:          689222   
Attachments:   ID                                     Name                       Type         Mime Type      
               d7c77cf9-a52a-4692-b9ec-ea1ec73d1f3a   credit_risk_training.csv   data_asset   text/csv      
                  


In [19]:
result = ! cpdctl asset get --project-id {PROJECT_ID} --asset-id {DATA_ASSET_ID} --output json --jmes-query "attachments[0].id" --raw-output
DATA_ATTACHMENT_ID = result.s
print("Data asset attachment ID is: {}".format(DATA_ATTACHMENT_ID))

Data asset attachment ID is: d7c77cf9-a52a-4692-b9ec-ea1ec73d1f3a


In [20]:
! cpdctl asset attachment download --project-id {PROJECT_ID} --asset-id {DATA_ASSET_ID} --attachment-id {DATA_ATTACHMENT_ID} --output-file credit_risk_training.csv

...
544.00 KiB / 673.07 KiB [------------------------------->_______] 80.82% ? p/s ?OK
Output written to credit_risk_training.csv


### Upload a new data asset

Clean up the existing "car_rental_training_data.csv" data assets

In [21]:
result = ! cpdctl asset search --project-id {PROJECT_ID} --type-name data_asset --query "asset.name:car_rental_training_data.csv" --output json --jmes-query "results[*].metadata.asset_id" --raw-output
DATA_ASSET_IDS = json.loads(result.s)
for data_asset_id in DATA_ASSET_IDS:
    print("Deleting data asset with ID: {}".format(data_asset_id))
    ! cpdctl asset delete --project-id {PROJECT_ID} --asset-id {data_asset_id}

Deleting data asset with ID: 80767054-a472-4e7b-ab19-89fdea8592c8
...
OK


Download another training data set from github 

In [22]:
! curl https://raw.githubusercontent.com/pmservice/wml-sample-models/master/spark/cars-4-you/data/car_rental_training_data.csv -o car_rental_training_data.csv
! wc -l car_rental_training_data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 79518  100 79518    0     0   129k      0 --:--:-- --:--:-- --:--:--  129k
     486 car_rental_training_data.csv


Create a new data asset in the project from the downloaded file "car_rental_training_data.csv" 

In [23]:
! cpdctl asset data-asset upload --file car_rental_training_data.csv --project-id {PROJECT_ID} --progress true --tag "cpdctl-demo" --mime "text/csv"

...
77.87 KiB / 77.65 KiB [---------------------------------------] 100.28% ? p/s 0s
                  
ID:            7b3720e3-1d13-4ea3-a76a-79ada7ff11e1   
Name:          car_rental_training_data.csv   
Created:       2021-07-15T11:33:21.000Z   
Description:      
Type:          data_asset   
State:         available   
Tags:          [cpdctl-demo]   
Size:          79518   
Attachments:   ID                                     Name                           Type         Mime Type      
               84b6baaa-0cb9-4d09-854c-ca85361c155b   car_rental_training_data.csv   data_asset   text/csv      
                  


In [24]:
result = ! cpdctl asset search --project-id {PROJECT_ID} --type-name data_asset --query "asset.name:car_rental_training_data.csv" --output json --jmes-query "results[0].metadata.asset_id" --raw-output
NEW_DATA_ASSET_ID = result.s
print("'car_rental_training_data.csv' data asset ID is: {}".format(NEW_DATA_ASSET_ID))

'car_rental_training_data.csv' data asset ID is: 7b3720e3-1d13-4ea3-a76a-79ada7ff11e1


In [25]:
! cpdctl asset search --project-id {PROJECT_ID} --type-name data_asset --query "*:*"

...
ID                                     Name                           Created                    Description   Type         State       Tags            Size   
bbeea775-b464-4abf-b2a2-17c31970cc47   credit_risk_training.csv       2021-07-15T10:19:26.000Z                 data_asset   available   [cpdctl-demo]   689222   
7b3720e3-1d13-4ea3-a76a-79ada7ff11e1   car_rental_training_data.csv   2021-07-15T11:33:21.000Z                 data_asset   available   [cpdctl-demo]   79518   


Promote the new data asset to "cpdctl-demo-space" space

In [26]:
! cpdctl space list

...
ID                                     Name           Created                    Description   State    Tags   
9001e03a-e3b9-4ddf-88db-063651d735b4   qa-space       2021-06-21T11:11:22.988Z                 active   []   
715f068a-a8c8-4093-a7f2-15ab111c21e5   prod-space     2021-06-21T11:12:12.958Z                 active   []   
a72bfe88-5fc9-4323-90ad-53d94511d120   fvt-space      2021-06-28T12:50:33.447Z                 active   []   
cdd90e91-792c-4834-a26a-ac979b998fd8   export-space   2021-07-09T12:02:55.754Z                 active   []   
be32f9b8-702f-444e-9902-754464f39853   jm-space       2021-07-15T10:40:36.849Z                 active   []   


Create a new space

In [27]:
! cpdctl space create --name cpdctl-demo-space

...
              
ID:        4265aad6-f75c-4ae9-b6ae-ea432822fde6   
Name:      cpdctl-demo-space   
Created:   2021-07-15T11:34:42.285Z   
State:     active   
Tags:      []   


In [28]:
! cpdctl space list

...
ID                                     Name                Created                    Description   State    Tags   
9001e03a-e3b9-4ddf-88db-063651d735b4   qa-space            2021-06-21T11:11:22.988Z                 active   []   
715f068a-a8c8-4093-a7f2-15ab111c21e5   prod-space          2021-06-21T11:12:12.958Z                 active   []   
a72bfe88-5fc9-4323-90ad-53d94511d120   fvt-space           2021-06-28T12:50:33.447Z                 active   []   
cdd90e91-792c-4834-a26a-ac979b998fd8   export-space        2021-07-09T12:02:55.754Z                 active   []   
be32f9b8-702f-444e-9902-754464f39853   jm-space            2021-07-15T10:40:36.849Z                 active   []   
4265aad6-f75c-4ae9-b6ae-ea432822fde6   cpdctl-demo-space   2021-07-15T11:34:42.285Z                 active   []   


Get the ID of cpdctl-demo-space

In [29]:
result = ! cpdctl space list --output json --jmes-query "resources[5].metadata.id" --raw-output
DEV_SPACE_ID = result.s
print("Space ID is: {}".format(DEV_SPACE_ID))

Space ID is: 4265aad6-f75c-4ae9-b6ae-ea432822fde6


Clean up existing data asset

In [30]:
result = ! cpdctl asset search --space-id {DEV_SPACE_ID} --type-name data_asset --query "asset.name:car_rental_training_data.csv" --output json --jmes-query "results[*].metadata.asset_id" --raw-output
DATA_ASSET_IDS = json.loads(result.s)
for data_asset_id in DATA_ASSET_IDS:
    print("Deleting data asset with ID: {}".format(data_asset_id))
    ! cpdctl asset delete --space-id {DEV_SPACE_ID} --asset-id {data_asset_id}

In [31]:
PROMOTE_BODY = {
    "mode": 0,
    "space_id": DEV_SPACE_ID,
    "metadata": {
        "tags": ["cpdctl-demo", "promoted-{}]".format(PROJECT_ID[0:8])]
    }
}
PROMOTE_BODY_JSON = json.dumps(PROMOTE_BODY)

! cpdctl asset promote --project-id {PROJECT_ID} --asset-id {NEW_DATA_ASSET_ID} --request-body '{PROMOTE_BODY_JSON}'

...
OK


In [32]:
! cpdctl asset search --space-id {DEV_SPACE_ID} --type-name asset --query "*:*"

...
ID                                     Name                           Created                    Description   Type         State       Tags                               Size   
6fff9b89-324e-405d-b6d8-0b14e89aa214   car_rental_training_data.csv   2021-07-15T11:35:09.000Z                 data_asset   available   [cpdctl-demo promoted-e655370d]]   79518   


### Train and store model in the notebook

List jobs in the project

In [33]:
result = ! cpdctl asset search --project-id {PROJECT_ID} --type-name job --output json --query "asset.name:train-scikit-model-job" --jmes-query "results[0].metadata.asset_id" --raw-output
EXISTING_JOB_ID = result.s
if EXISTING_JOB_ID and EXISTING_JOB_ID != "null":
    print("Deleting job with ID: {}".format(EXISTING_JOB_ID))
    ! cpdctl job delete --project-id {PROJECT_ID} --job-id {EXISTING_JOB_ID}
    

List all the environments in your project, filter them by their display name and get the ID of the environment in which your notebook will be run

In [34]:
environment_name = "Default Python 3.7"
query_string = "(resources[?entity.environment.display_name == '{}'].metadata.asset_id)[0]".format(environment_name)
result = ! cpdctl environment list --project-id {PROJECT_ID} --output json -j "{query_string}" --raw-output
env_id = result.s
print("environment id: {}".format(env_id))

environment id: jupconda37oce-e655370d-4513-45f9-b4c6-c45692044773



Upload the notebook

In [35]:
remote_file_path = "notebook/train-scikit-model-to-predict-boston-house-prices.ipynb"
local_file_path = "train-scikit-model-to-predict-boston-house-prices.ipynb"

! cpdctl asset file upload --path {remote_file_path} --file {local_file_path} --project-id {PROJECT_ID}

...
OK


Create a notebook asset

In [36]:
file_name = "train-scikit-model-to-predict-boston-house-prices.ipynb"
runtime = {
    'environment': env_id
}
runtime_json = json.dumps(runtime)

result = ! cpdctl notebook create --file-reference {remote_file_path} --name {file_name} --project {PROJECT_ID} --runtime '{runtime_json}' --output json -j "metadata.asset_id" --raw-output
NOTEBOOK_ID = result.s
print("'train-scikit-model-to-predict-boston-house-prices' notebook ID is: {}".format(NOTEBOOK_ID))

'train-scikit-model-to-predict-boston-house-prices' notebook ID is: 07f0b515-f125-49bc-8628-719c98c889d7


In [37]:
! cpdctl asset search --project-id {PROJECT_ID} --type-name notebook --query "*:*"

...
ID                                     Name                                                 Created                    Description   Type       State       Tags         Size   
91248743-3524-4b0f-878d-3aeef88c5fa5   Sample notebook                                      2021-07-15T09:29:11.000Z                 notebook   available   [notebook]   207   
07f0b515-f125-49bc-8628-719c98c889d7   train-scikit-model-to-predict-boston-house-prices…   2021-07-15T11:35:48.000Z                 notebook   available   [notebook]   36283   


In order to run the notebook, you must fill it in with your user name, password, url and space ID

In [38]:
JOB = {
    "asset_ref": NOTEBOOK_ID,
    "name": "train-scikit-model-job",
    "configuration": {
        "env_id": env_id
    }
}
JOB_JSON = json.dumps(JOB)
result = ! cpdctl job create --project-id {PROJECT_ID} --job '{JOB_JSON}' --output json --jmes-query "metadata.asset_id" --raw-output
JOB_ID = result.s
print("'train-scikit-model-job' job ID is: {}".format(JOB_ID))

'train-scikit-model-job' job ID is: 08539857-20f3-4d4a-ac14-846e75e34e04


Trigger job run

In [39]:
JOB_RUN = {
    "job_run": {}
}
JOB_RUN_JSON = json.dumps(JOB_RUN)
result = ! cpdctl job run create --project-id {PROJECT_ID} --job-id {JOB_ID} --job-run '{JOB_RUN_JSON}' --async --output json --jmes-query 'metadata.asset_id'
RUN_ID = result.s
print("The new run ID for 'train-scikit-model-job' job: {}".format(RUN_ID))

The new run ID for 'train-scikit-model-job' job: "afc59353-35c2-4524-a579-673ae6750253"


Get notebook run status

In [40]:
! cpdctl job run get --project-id {PROJECT_ID} --job-id {JOB_ID} --run-id {RUN_ID} --output json --jmes-query "entity.job_run.state" --raw-output

Running


In [42]:
! cpdctl job run logs --project-id {PROJECT_ID} --job-id {JOB_ID} --run-id {RUN_ID}

...

Cell 9:

     |▌                               | 30 kB 2.0 MB/s eta 0:00:01
     |██                              | 112 kB 1.0 MB/s eta 0:00:02
     |███                             | 174 kB 1.0 MB/s eta 0:00:02
     |████▏                           | 245 kB 1.0 MB/s eta 0:00:02
     |█████▌                          | 327 kB 1.0 MB/s eta 0:00:02
     |███████                         | 409 kB 1.0 MB/s eta 0:00:02
     |███████▍                        | 440 kB 1.0 MB/s eta 0:00:02
     |█████████▎                      | 552 kB 1.0 MB/s eta 0:00:02
     |██████████▍                     | 614 kB 1.0 MB/s eta 0:00:02
     |███████████▊                    | 696 kB 1.0 MB/s eta 0:00:02
     |████████████▋                   | 747 kB 1.0 MB/s eta 0:00:02
     |██████████████                  | 829 kB 1.0 MB/s eta 0:00:02
     |███████████████▏                | 901 kB 1.0 MB/s eta 0:00:01
     |████████████████▎               | 962 kB 1.0 MB/s eta 0:00:01
     |█████████████████▎           

In [43]:
result = ! cpdctl asset search --space-id {DEV_SPACE_ID} --type-name wml_model --query "asset.name:External scikit model - updated" --output json --jmes-query "results[0].metadata.asset_id" --raw-output
MODEL_ID = result.s
print("'boston-house-prices-prediction-model' model ID is: {}".format(MODEL_ID))

'boston-house-prices-prediction-model' model ID is: ba8ae04b-a049-4346-a654-be364a1d0db9


In [45]:
! cpdctl ml model list --space-id {DEV_SPACE_ID}

...
ID                                     Name                                   Created                    Type                Tags   
ba8ae04b-a049-4346-a654-be364a1d0db9   boston-house-prices-prediction-model   2021-08-11T13:42:13.002Z   scikit-learn_0.23   []   


Export space assets

In [45]:
EXPORT = {
    'asset_ids': [MODEL_ID]
}
EXPORT_JSON = json.dumps(EXPORT)
result = ! cpdctl asset export start --space-id {DEV_SPACE_ID} --assets '{EXPORT_JSON}' --name dev-space-assets --output json --jmes-query "metadata.id"
EXPORT_ID = result.s
print("The new export with ID: {}".format(EXPORT_ID))

The new export with ID: "5a0e7988-253f-4d48-b780-ae345b10069e"


In [46]:
! cpdctl asset export get --space-id {DEV_SPACE_ID} --export-id {EXPORT_ID}

...
              
ID:        5a0e7988-253f-4d48-b780-ae345b10069e   
Name:      dev-space-assets   
Created:   2021-07-15T11:44:09.475Z   
State:     completed   


In [47]:
! cpdctl asset export download --space-id {DEV_SPACE_ID} --export-id {EXPORT_ID} --output-file dev-space-assets.zip

...
OK
Output written to dev-space-assets.zip


In [48]:
! unzip -l dev-space-assets.zip 

Archive:  dev-space-assets.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
       44  08-11-2021 13:44   deflate.log
      476  08-11-2021 13:44   assettypes/omrs_relationship_message.json
      344  08-11-2021 13:44   assettypes/omrs_entity.json
      469  08-11-2021 13:44   assettypes/data_asset.json
      479  08-11-2021 13:44   assettypes/omrs_relationship.json
      251  08-11-2021 13:44   assettypes/policy_transform.json
      206  08-11-2021 13:44   assettypes/data_flow.json
      326  08-11-2021 13:44   assettypes/connection.json
    16789  08-11-2021 13:44   assettypes/wml_function.json
     2545  08-11-2021 13:44   assettypes/wml_remote_training_system.json
    43577  08-11-2021 13:44   assettypes/wml_pipeline.json
    40878  08-11-2021 13:44   assettypes/wml_deployment_job_definition.json
      546  08-11-2021 13:44   assettypes/connection_credentials.json
      543  08-11-2021 13:44   assettypes/environment.json
      494  08-11-2021 13:44   assett

### Create a new QA space and import assets

Ensure there is no QA space

In [49]:
result = ! cpdctl space list --output json --jmes-query "resources[?entity.name == 'cpdctl-demo-qa-space'].metadata.id"
SPACE_IDS = json.loads(result.s)
if SPACE_IDS:
    for space_id in SPACE_IDS:
        result = ! cpdctl ml deployment list --space-id {space_id} --output json --jmes-query "resources[*].metadata.id"
        DEPLOYMENT_IDS = json.loads(result.s)
        if DEPLOYMENT_IDS:
            for deployment_id in DEPLOYMENT_IDS:
                print('Deleting deployment with ID: {}'.format(space_id))
                ! cpdctl ml deployment delete --space-id {space_id} --deployment-id {deployment_id}
        print('Deleting space with ID: {}'.format(space_id))
        ! cpdctl space delete --space-id {space_id}

Create a new QA space

In [50]:
result = ! cpdctl space create --name cpdctl-demo-qa-space --output json --jmes-query "metadata.id" --raw-output
QA_SPACE_ID = result.s
print("The new 'cpdctl-demo-qa-space' space ID is: {}".format(QA_SPACE_ID))

The new 'cpdctl-demo-qa-space' space ID is: 2c83e24c-c36e-41d3-ad94-c2f5ce6fa723


Import assets exported from DEV space

In [51]:
result = ! cpdctl asset import start --space-id {QA_SPACE_ID} --import-file dev-space-assets.zip --output json --jmes-query "metadata.id" --raw-output
QA_IMPORT_ID = result.s
print("The new import ID is: {}".format(QA_IMPORT_ID))

The new import ID is: 07b74a2b-799b-4de3-8bb5-7fd9783a05e7


In [52]:
! cpdctl asset import get --space-id {QA_SPACE_ID} --import-id {QA_IMPORT_ID}

...
              
ID:        07b74a2b-799b-4de3-8bb5-7fd9783a05e7   
Created:   2021-07-15T11:45:03.886Z   
State:     completed   


In [53]:
! cpdctl ml model list --space-id {QA_SPACE_ID}

...
ID                                     Name                                   Created                    Type                Tags   
42f3fdcb-9304-44a3-b711-0a9761dc6f5d   boston-house-prices-prediction-model   2021-08-11T13:45:23.002Z   scikit-learn_0.23   []   


In [54]:
result = ! cpdctl ml model list --space-id {QA_SPACE_ID} --output json --jmes-query "resources[0].metadata.id" --raw-output
QA_MODEL_ID = result.s
print("QA model ID is: {}".format(QA_MODEL_ID))

QA model ID is: 42f3fdcb-9304-44a3-b711-0a9761dc6f5d


Create online deployment

In [55]:
ASSET_JSON = json.dumps({"id": QA_MODEL_ID})
ONLINE_JSON = json.dumps({})

! cpdctl ml deployment create --space-id {QA_SPACE_ID} --asset '{ASSET_JSON}' --online '{ONLINE_JSON}'

...
              
ID:        81c95d17-3a55-4c4d-a6e7-26bbbb459f43   
Name:         
Created:   2021-07-15T11:45:36.826Z   
State:     ready   
Tags:      []   


In [56]:
! cpdctl ml deployment list  --space-id {QA_SPACE_ID}

...
ID                                     Name   Created                    State   Tags   
81c95d17-3a55-4c4d-a6e7-26bbbb459f43          2021-07-15T11:45:36.826Z   ready   []   


### Author

Rafał Bigaj, System Architect with long successful record of building and leading teams. Broad and practical knowledge in the area of cloud computing, machine learning and distributed systems development. 

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.